<a href="https://colab.research.google.com/github/p1306/Leaf-Analysis-Project/blob/main/LeafAgeEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os

In [ ]:
# Define paths
base_dir = '/content/drive/MyDrive/LeafAge'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# ImageDataGenerator for data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Load the training set
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Resize images
    batch_size=10,  # Number of images to process at once
    class_mode='categorical'  # We have 4 categories (T1, T2, T3, T4)
)

Found 39 images belonging to 4 classes.


In [ ]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 output classes (T1, T2, T3, T4)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=10,  # Adjust based on your dataset size
    epochs=15  # Adjust based on accuracy and training time
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 144ms/step - accuracy: 0.2416 - loss: 2.2776
Epoch 2/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 228ms/step - accuracy: 0.2649 - loss: 1.4431
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.5093 - loss: 1.2344
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 138ms/step - accuracy: 0.5500 - loss: 0.9531
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.5482 - loss: 0.9526
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.6454 - loss: 0.9011
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - accuracy: 0.5880 - loss: 1.1323
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - accuracy: 0.4395 - loss: 0.9190
Epoch 9/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.6916 - loss: 0.7347
Epoch 10/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 141ms/step - accuracy: 0.5530 - loss: 0.8431
Epoch 11/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.6596 - loss: 0.7312
Epoch 12/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 228ms/step - accuracy: 0.6477 - loss: 0.7245
Epoch 13/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accurac

In [ ]:
# Function to load and preprocess a single image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = load_img(img_path, target_size=target_size)  # Use load_img from keras.preprocessing
    img_array = img_to_array(img)  # Use img_to_array from keras.preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
# Load and predict uncategorized test images
test_image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir) if img.endswith(('png', 'jpg', 'jpeg'))]

In [ ]:
for img_path in test_image_paths:
    img_array = load_and_preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)  # Get the class with highest probability

    # Mapping the predicted class index to the corresponding age category
    class_names = ['T1 (1-2 days)', 'T2 (3-4 days)', 'T3 (5-7 days)', 'T4 (7+ days)']
    predicted_label = class_names[predicted_class[0]]

    print(f"Image {img_path} is predicted as: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Image /content/drive/MyDrive/LeafAge/test/Screenshot 2024-09-05 225437.png is predicted as: T4 (7+ days)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image /content/drive/MyDrive/LeafAge/test/Screenshot 2024-09-05 225504.png is predicted as: T1 (1-2 days)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image /content/drive/MyDrive/LeafAge/test/Screenshot 2024-09-05 225526.png is predicted as: T1 (1-2 days)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image /content/drive/MyDrive/LeafAge/test/Screenshot 2024-09-05 225617.png is predicted as: T4 (7+ days)


In [ ]:
model.save('leaf_age_model.h5')


In [ ]:
from google.colab import files
files.download('/content/leaf_age_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>